In [9]:
import pandas as pd
import pandas as pd, numpy as np
from pathlib import Path
import fsspec

Question 1

In [2]:
def station_data(station_id):
    
    # Receives a GHCN Station ID string and returns record daily max,
    # average daily max, record daily min and average daily min for 
    # the period 1991-2020
    
    begin_date = "1-Jan-1990"
    end_date = "31-Dec-2020"

    download_string = "s3://noaa-ghcn-pds/csv/by_station/" + str(station_id) + ".csv"
    df = pd.read_csv( 
                     download_string,
                     storage_options={"anon": True},  # passed to `s3fs.S3FileSystem`
                     dtype={'Q_FLAG': 'object', 'M_FLAG': 'object'},
                     parse_dates=['DATE']
                     ).set_index('DATE')
    
    # Filtering for date
    df = df[(df.index >= pd.to_datetime(begin_date)) & (df.index <= pd.to_datetime(end_date))]

    df_daily_min = df[df['ELEMENT'] == 'TMIN']
    df_daily_max = df[df['ELEMENT'] == 'TMAX']
    
    min_min = df_daily_min['DATA_VALUE'].min()
    max_max = df_daily_min['DATA_VALUE'].max()
    min_average = df_daily_min['DATA_VALUE'].mean()
    max_average = df_daily_max['DATA_VALUE'].mean()

    result = pd.DataFrame({"record_min_temp": min_min, 
                           "average_min_temp": min_average, 
                           "average_max_temp": max_average,
                           "record_max_temp": max_max},
                           index = [0])    

    return result



In [3]:
df = station_data("USC00118740")

/tmp/ipykernel_43363/1332965351.py:11: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(


In [4]:
df

,record_min_temp,average_min_temp,average_max_temp,record_max_temp
0,-317,58.815232,168.450098,256


Question 2

In [5]:
# Fill Between

# 1. Download city data

# 2. Groupby day of year and find max high, and mix low

# 3. Groupby day of year and take mean high and mean low

# 4. Pick a year and create a time series of high temps and low temps

# 5. Plot

In [10]:
S3_STATIONS_TXT   = "s3://noaa-ghcn-pds/ghcnd-stations.txt"
S3_INVENTORY_TXT  = "s3://noaa-ghcn-pds/ghcnd-inventory.txt"
S3_BY_STATION     = "s3://noaa-ghcn-pds/csv/by_station/{id}.csv"
STOR = {"anon": True}

OUTDIR = Path('../data'); OUTDIR.mkdir(parents=True, exist_ok=True)
OUT_PARQUET = OUTDIR / 'ghcn_il_top4_daily.parquet'
OUT_CSV = OUTDIR / 'ghcn_il_top4_daily.csv'
print('Output:', OUT_PARQUET.resolve())

Output: /home/weatherdev/ATMS Masters/ATMS523/3Module/Assignment/data/ghcn_il_top4_daily.parquet


In [19]:
stn_ids = pd.read_fwf('http://noaa-ghcn-pds.s3.amazonaws.com/ghcnd-stations.txt', header=None, infer_nrows=1000)
stn_ids.columns = ['ID','LAT','LON','ELEV','STATE','NAME','GSN','WBAN']

periods = pd.read_fwf('http://noaa-ghcn-pds.s3.amazonaws.com/ghcnd-inventory.txt', header=None, infer_nrows=1000)
periods.columns = ['ID','LAT','LON','ELEM','TiMIN','TiMAX']

merged_stns = pd.merge(stn_ids,periods,how='left',left_on='ID',right_on='ID')

In [24]:
merged_stns.head()

,ID,LAT_x,LON_x,ELEV,STATE,NAME,GSN,WBAN,LAT_y,LON_y,ELEM,TiMIN,TiMAX
0,ACW00011604,17.1167,-61.7833,10.1,NaN,ST JOHNS COOLIDGE FLD,NaN,NaN,17.1167,-61.7833,TMAX,1949.0,1949.0
1,ACW00011604,17.1167,-61.7833,10.1,NaN,ST JOHNS COOLIDGE FLD,NaN,NaN,17.1167,-61.7833,TMIN,1949.0,1949.0
2,ACW00011604,17.1167,-61.7833,10.1,NaN,ST JOHNS COOLIDGE FLD,NaN,NaN,17.1167,-61.7833,PRCP,1949.0,1949.0
3,ACW00011604,17.1167,-61.7833,10.1,NaN,ST JOHNS COOLIDGE FLD,NaN,NaN,17.1167,-61.7833,SNOW,1949.0,1949.0
4,ACW00011604,17.1167,-61.7833,10.1,NaN,ST JOHNS COOLIDGE FLD,NaN,NaN,17.1167,-61.7833,SNWD,1949.0,1949.0


In [20]:
merged_stns['STATE'].unique()

array([nan, 'AS', 'BH', 'BC', 'YT', 'NT', 'NU', 'AB', 'SK', 'MB', 'ON',
       'QC', 'NB', 'NS', 'PE', 'NL', 'MP', 'FM', 'GU', 'UM', 'PW', 'MH',
       'PR', 'SA', 'SD', 'CO', 'NE', 'AK', 'AL', 'AR', 'AZ', 'CA', 'TN',
       'CT', 'DC', 'DE', 'FL', 'GA', 'HI', 'IA', 'ID', 'IL', 'IN', 'KS',
       'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN', 'MO', 'MS', 'MT', 'NC',
       'ND', 'NH', 'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI',
       'SC', 'TX', 'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY', 'VI'],
      dtype=object)

In [37]:
filtered_stns = merged_stns[(merged_stns['STATE'] == 'VT') & (merged_stns['TiMIN'] < 1991) & (merged_stns['TiMAX'] > 2019)]

In [38]:
filtered_stns[filtered_stns['NAME'] == 'ROCHESTER']

,ID,LAT_x,LON_x,ELEV,STATE,NAME,GSN,WBAN,LAT_y,LON_y,ELEM,TiMIN,TiMAX
667628,USC00436893,43.8578,-72.8045,253.0,VT,ROCHESTER,NaN,NaN,43.8578,-72.8045,TMAX,1964.0,2020.0
667629,USC00436893,43.8578,-72.8045,253.0,VT,ROCHESTER,NaN,NaN,43.8578,-72.8045,TMIN,1964.0,2020.0
667630,USC00436893,43.8578,-72.8045,253.0,VT,ROCHESTER,NaN,NaN,43.8578,-72.8045,TOBS,1964.0,2020.0
667631,USC00436893,43.8578,-72.8045,253.0,VT,ROCHESTER,NaN,NaN,43.8578,-72.8045,PRCP,1928.0,2020.0
667632,USC00436893,43.8578,-72.8045,253.0,VT,ROCHESTER,NaN,NaN,43.8578,-72.8045,SNOW,1928.0,2020.0
667633,USC00436893,43.8578,-72.8045,253.0,VT,ROCHESTER,NaN,NaN,43.8578,-72.8045,SNWD,1928.0,2020.0


In [36]:
filtered_stns

array(['CHITTENDEN', 'CORINTH', 'CORNWALL', 'GILMAN', 'ISLAND POND',
       'MORRISVILLE-STOWE STATE AP', 'MT MANSFIELD', 'NEWPORT', 'PERU',
       'ROCHESTER', 'RUTLAND', 'SAINT JOHNSBURY', 'S HERO', 'S LINCOLN',
       'UNION VILLAGE DAM', 'W TOPSHAM', 'WOODSTOCK',
       'BURLINGTON INTL AP', 'MONTPELIER AP'], dtype=object)